构建一个论文Agent让大模型生成一篇完整的论文是可行的，只不过需要经过一系列的优化。

基本思想依然是“分而治之”，抑或是思维链。
算法设计如下：
1. 让agent在写论文开始的时候，确定整体思路、创新之处和论文框架。
2. 让agent每一次回答详细展开每一个部分的内容，比如第一次回答展开引言部分，第二次展开文献综述部分，以此类推，直到完成所有部分的撰写；
3. 在这个过程中设计一些算法初始化论文内容为空值，然后在回答过程中识别论文的内容，每回答一次就增加到论文正文部分。
4. 最后回答完毕就保存这个论文内容到word文档。

可以进一步优化的地方(to do)
- 增加一个优化写作的agent，每次写完一部分再优化一次
- 增加一个coding agent，根据论文的内容编写所有的代码并执行、自动纠错和优化
- 防AI检测，hhh


In [5]:
from tools.llm_api import *
import json
from tools.text2word import *

class EssayAgent:
    def __init__(self, angent_name, model, configs, temperature=0.9):
        self.name = angent_name
        self.model = model
        self.temperature = temperature
        system_prompt_path = configs["system_prompt_path"]
        with open(system_prompt_path, "r", encoding="utf-8") as f:
            self.system_prompt = f.read()
        self.conversations = [{"role": "system", "content": self.system_prompt}]
        self.total_essay = ""
        
    # 思考创新点
    
    # 生成回答
    def generate_response(self, conversations):
        ans = ""
        for char in get_llm_answer_converse(conversations=conversations, model=self.model, temperature=self.temperature):
            ans += char
            print(char, end="", flush=True)
        self.conversations.append({"role": "assistant", "content": ans})
        return ans
    
    # 执行论文撰写
    def write_essay(self, frame, epoch_num=5):
        prompt = f'''
        我的论文框架如下：
        {frame}
        '''
        self.conversations.append({"role": "user", "content": prompt})
        
        ans = ""
        write_num = 0
        while "[END]" not in ans:
            if write_num < epoch_num:
                print("\n" + "="*30 + f"第{write_num+1}次" + "="*30 + "\n")
                ans = self.generate_response(self.conversations)
                self.conversations.append({"role": "user", "content": "继续"})
                write_num += 1
            else:
                break
        
        text2word(self.total_essay, "essay.docx")
        return self.conversations

    # 提取论文的所有内容并保存为markdown文件
    
    
# 论文框架
frame = '''
主题：Transformer模型的创新与应用
1. 引言
2. 相关工作
3. 模型
'''

model = "gpt-4o"
# 打开config路径文件：configs/essay_agent_config.json
with open("configs/essay_agent_config.json", "r", encoding="utf-8") as f:
    configs = json.load(f)
essay_agent = EssayAgent(angent_name="essay_agent", model=model, configs=configs)
essay_agent.write_essay(frame)


==============================第1次==============================

根据你给出来的论文框架，首先我们确定一个具体的、细分的主题，并且给出一个更具有创新性的题目，然后为每个章节的标题写一个该部分的主要研究内容。

### 主题：Transformer模型的创新与应用

#### 创新性题目：基于多任务学习的自适应Transformer模型研究及其在复杂自然语言处理任务中的应用

### 论文框架与研究内容：

1. **引言**
    - **主要研究内容：** 
        - 背景介绍：Transformer模型的发展历史与现状。
        - 动机与问题陈述：现有Transformer模型在复杂自然语言处理任务中的局限性。
        - 研究目标与创新性贡献：介绍提出一种基于多任务学习的自适应Transformer模型，并说明其创新点及在不同任务中的应用前景。

2. **相关工作**
    - **主要研究内容：**
        - Transformer模型的基本原理与重要变体（如BERT、GPT、T5等）。
        - 多任务学习在自然语言处理中的应用。
        - 自适应模型技术与优势。
        - 综合评述现有技术和模型存在的问题，并为本文的研究提供依据。

3. **模型**
    - **主要研究内容：**
        - 模型架构：详细描述提出的基于多任务学习的自适应Transformer模型的结构。
        - 自适应机制：设计实现如何通过多任务学习进行模型的自适应调整。
        - 训练策略与算法：详细介绍模型的训练流程与关键算法，包括如何处理多任务学习中的任务权重分配问题。
        - 性能优化：针对大规模数据与复杂任务提出的优化方法。

请指定你想要撰写的部分，从具体内容开始详细撰写。
==============================第2次==============================

### 1. 引言

#### 背景介绍

Transformer模型自其提出以来（Vaswani等，2017），在自然语言处理（NLP）领域引发了广泛关注，并逐渐成为一种主流的深度学习架

[{'role': 'system',
  'content': '# 你的角色\n具有十年经验的深度学习专家\n\n# 你的任务\n你需要根据我给出来的论文框架,一步步撰写论文的所有内容\n\n# 你的工作流\n你必须严格按照以下步骤解决我的问题：\n1. 你的第一次回答是这样的：根据我给出来的论文框架,首先确定一个具体的、细分的主题, 并且给出一个更具有创新性的题目，然后为每个章节的标题写一个该部分的主要研究内容；\n你接下来的所有回答，都是根据你第一次回答给出来的框架中的某个标题，撰写该部分的详细内容；\n- 注意：你的每次回答都只需要包含一个部分的内容,不要包含多个部分的内容，当我说继续的时候，你再继续下一个部分的内容\n3. 当所有论文撰写结束的时候,你只需要回复一个"[END]"即可。\n\n# 注意\n- 你必须时刻记得，如果我的需求完毕之后，你需要第一时间回复"[END]"表示工作完成\n- 你的论文必须包含创新点，思考我的论文框架有哪些地方可以做创新，然后进行创新性的撰写\n- 你不需要向我确认任何事情，直接开始写论文即可\n'},
 {'role': 'user',
  'content': '\n        我的论文框架如下：\n        \n主题：Transformer模型的创新与应用\n1. 引言\n2. 相关工作\n3. 模型\n\n        '},
 {'role': 'assistant',
  'content': '根据你给出来的论文框架，首先我们确定一个具体的、细分的主题，并且给出一个更具有创新性的题目，然后为每个章节的标题写一个该部分的主要研究内容。\n\n### 主题：Transformer模型的创新与应用\n\n#### 创新性题目：基于多任务学习的自适应Transformer模型研究及其在复杂自然语言处理任务中的应用\n\n### 论文框架与研究内容：\n\n1. **引言**\n    - **主要研究内容：** \n        - 背景介绍：Transformer模型的发展历史与现状。\n        - 动机与问题陈述：现有Transformer模型在复杂自然语言处理任务中的局限性。\n        - 研究目标与创新性贡献：介绍提出一种基于多任务学习的自适应Transformer模型，并说